Import Part

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import stats
from pandas import DataFrame

Const

In [2]:
FILES_FOLDER_PATH = 'files_cambridgeshire_bedfordshire'
FILE_NAME = 'cambridgeshire_bedfordshire.csv'

Functions

In [3]:
def remove_outliers(df: DataFrame):
    Q1 = np.percentile(df['wqi'], 25,
                   interpolation = 'midpoint')
 
    Q3 = np.percentile(df['wqi'], 75,
                    interpolation = 'midpoint')
    IQR = Q3 - Q1
    ''' Removing the Outliers '''
    df = df[(df['wqi'] > (Q1 - 1.5 * IQR)) & (df['wqi'] < (Q3 + 1.5 * IQR))]
    return df

Loading dataset

In [4]:
df = pd.read_csv(os.path.join(FILES_FOLDER_PATH, FILE_NAME), header=0, index_col=0)
df.index = pd.to_datetime(df.index)
df = df.sort_values('date', ascending=True)

Removing outliers

In [5]:
df = remove_outliers(df)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Sliding window

In [9]:
for step in range(1, 50):
    values = DataFrame(df.values)
    dataframe = DataFrame()
    dataframe = pd.concat([values.shift(i) for i in range(step, -1, -1)], axis=1)
    columns = []
    for i in range(step, -1, -1):
        t = f"t-{i}" if i != 0 else "t"
        columns.extend([f"{t}-x{j}" for j in range(1, len(df.columns) + 1)])

    columns = columns[:-1] + ["y"]
    dataframe.columns = columns 
    dataframe = dataframe.dropna()

    X = dataframe.values[:, :-1]
    y = dataframe.values[:, -1]

    train_max_index = int(len(X) * 0.8)
    Xtrain = X[: train_max_index]
    ytrain = y[: train_max_index]
    Xtest = X[train_max_index:]
    ytest= y[train_max_index:]

    mdl = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=23)
    mdl.fit(Xtrain, ytrain)

    p = mdl.predict(Xtest)

    error = mean_squared_error(ytest, p)
    print('Lag size: %d, RMSE %.5f' % (step, error))

Lag size: 1, RMSE 7.58245
Lag size: 2, RMSE 7.86252
Lag size: 3, RMSE 7.85290
Lag size: 4, RMSE 7.80802
Lag size: 5, RMSE 8.14538
Lag size: 6, RMSE 8.37765
Lag size: 7, RMSE 8.39950
Lag size: 8, RMSE 8.39740
Lag size: 9, RMSE 8.60404
Lag size: 10, RMSE 8.44485
Lag size: 11, RMSE 8.36886
Lag size: 12, RMSE 8.50237
Lag size: 13, RMSE 8.84419
Lag size: 14, RMSE 8.61914
Lag size: 15, RMSE 8.81657
Lag size: 16, RMSE 8.85953
Lag size: 17, RMSE 8.91868
Lag size: 18, RMSE 8.79580
Lag size: 19, RMSE 8.80082
Lag size: 20, RMSE 8.84684
Lag size: 21, RMSE 8.80013
Lag size: 22, RMSE 8.85301
Lag size: 23, RMSE 9.02446
Lag size: 24, RMSE 9.16742
Lag size: 25, RMSE 9.06580
Lag size: 26, RMSE 8.98921
Lag size: 27, RMSE 9.23500
Lag size: 28, RMSE 9.20613
Lag size: 29, RMSE 9.29253
Lag size: 30, RMSE 9.23570
Lag size: 31, RMSE 9.07394
Lag size: 32, RMSE 9.36435
Lag size: 33, RMSE 9.24954
Lag size: 34, RMSE 9.15475
Lag size: 35, RMSE 9.11571
Lag size: 36, RMSE 9.21065
Lag size: 37, RMSE 9.12862
Lag size: 